# Сравнение 1NN и случайного леса

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

In [1]:
import numpy as np
import sklearn
from sklearn import ensemble, metrics, datasets

## Подготовка данных

В этом задании будет использоваться датасет digits из sklearn.datasets. 

In [2]:
#Загружаем датасет 
digits = datasets.load_digits()

#Выделяем матрицу объектов-признаков и вектор ответов
data = digits.data
target = digits.target

In [3]:
#Просмотрим размерность данных
print('Набор данных содержит {} объектов и {} признаков'.format(data.shape[0], data.shape[1]))

Набор данных содержит 1797 объектов и 64 признаков


In [4]:
#Теперь узнаем, какие уникальные метки классов есть в наборе данных
print(set(target))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


Таким образом, как можно было предположить из описания датасета, в нем есть 10 различных классов - каждый класс представляет одну из цифр. 

In [5]:
#Разбиваем данные на обучение и тест

from sklearn.model_selection import train_test_split
data_train, data_test, target_train, target_test = train_test_split = train_test_split(data, target,
                                                        test_size = 0.25, random_state = 0, stratify = target)

In [6]:
for tar in set(target):
    print('Набор данных содержит {} объектов класса {}'.format(np.sum(target == tar), tar))

Набор данных содержит 178 объектов класса 0
Набор данных содержит 182 объектов класса 1
Набор данных содержит 177 объектов класса 2
Набор данных содержит 183 объектов класса 3
Набор данных содержит 181 объектов класса 4
Набор данных содержит 182 объектов класса 5
Набор данных содержит 181 объектов класса 6
Набор данных содержит 179 объектов класса 7
Набор данных содержит 174 объектов класса 8
Набор данных содержит 180 объектов класса 9


Учитывая, что выборка сбалансированная - количество объектов, относящихся к каждому из классов, примерно одинаково, в данной задаче мы можем использовать accuracy в качестве метрики качества.

## Задание №1

Для начала обучим RandomForestClassifier с количеством деревьев, равным 1000

In [7]:
%%time

#Создаем объект случайного леса и обучаем его на соответствующей выборке
rf_est = ensemble.RandomForestClassifier(n_estimators=1000)
rf_est.fit(data_train, target_train)

#Предсказываем метки классов на тестовой выборке
rf_predicts = rf_est.predict(data_test)

#Считаем accuracy на тестовой выборке по кросс-валидации
rf_acc_score = metrics.accuracy_score(target_test, rf_predicts)

Wall time: 5.29 s


In [8]:
print('Качество случайного леса на тестовой выборке: {}'.format(round(rf_acc_score, 3)))

Качество случайного леса на тестовой выборке: 0.976


In [9]:
with open ("kNN_vs_RF_2.txt", 'w') as ans_2:
    ans_2.write(str(1 - rf_acc_score))
ans_2.close()

## Задание №2

Теперь реализуем метод одного ближайшего соседа с евклидовой метрикой для задачи классификации.

In [10]:
from sklearn import neighbors

In [11]:
%%time

#Создаем объект классификатора и проводим обучение на соответствующей выборке
neigh = neighbors.KNeighborsClassifier(n_neighbors = 1, metric = 'euclidean')
neigh.fit(data_train, target_train)

#Предсказываем метки классов на тестовой выборке
neigh_predicts = neigh.predict(data_test)

#Вычисляем accuracy на тесте
neigh_score = metrics.accuracy_score(target_test, neigh_predicts)

Wall time: 117 ms


In [12]:
print('Качество 1NearestNeighbor на тестовой выборке: {}'.format(round(neigh_score, 3)))

Качество 1NearestNeighbor на тестовой выборке: 0.982


In [13]:
with open("knn_answer.txt", 'w') as ans_1:
    ans_1.write(str(1 - neigh_score))
ans_1.close()

### Выводы по заданию

Итак, мы видим, что на данном наборе данных RandomForestClassifier и kNearestNeighbors показывают практически одинаковые результаты, однако примечательно то, что kNN потребовалось всего 117ms против 5.29s для RandomForestClassifier, что примерно в 45 раз быстрее! Это любопытный результат, который показывает, что на некоторых датасетах лучшее решение - самое простое. Конечно, нельзя делать серьезных выводов опираясь только на сравнение качества методов на данном наборе данных, ибо это всего лишь тренировочный набор. В этом датасете классы сбалансированны и объекты каждого класса хорошо отличимы друг от друга (если бы объекты разных классов были перемешаны в пространстве признаков, метод ближайшего соседа точно не показал бы такое высокое качество на тесте). Конечно, случайный лес куда более гибкий и универсальный метод, но данный пример показывает, что порой простые методы тоже показывают очень хорошие результаты.